In [12]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os


con = db.connect(database="../db.sqlite")

In [3]:

excelFileName = 'big-mac-2021-07-01.xls'
if not os.path.isfile(excelFileName):
    url = "https://github.com/TheEconomist/big-mac-data/blob/ef3a70d3589a4b0d82f2d753c47364a837ebd750/output-data/" + excelFileName + "?raw=true"
    print("Downloading: " + url)
    urllib.request.urlretrieve(url, excelFileName)
    print("Download complete")



xl = pd.ExcelFile(excelFileName)


df = pd.DataFrame()

for sheetName in xl.sheet_names:
        
    date = datetime.strptime(sheetName,"%b%Y")
       
    df2 = xl.parse(sheetName)
    
    
    df2["date"] = date
    
    
    df = pd.concat([df,df2])  

Downloading: https://github.com/TheEconomist/big-mac-data/blob/ef3a70d3589a4b0d82f2d753c47364a837ebd750/output-data/big-mac-2021-07-01.xls?raw=true
Download complete


In [9]:
##### insert countries and currencies #####

dfCurrencies = df.groupby("currency_code").first().reset_index()[["currency_code"]]
dfCurrencies = dfCurrencies.rename(columns={"currency_code": "CurrencyCode"})
dfCurrencies["Name"] = dfCurrencies["CurrencyCode"]

try:
    dfCurrencies.to_sql("Currency", con,if_exists="append", index=False)
except Exception as e:
    print(e)




dfCountries = df.groupby("iso_a3").first().reset_index()[["iso_a3", "Country", "currency_code"]]
dfCountries = dfCountries.rename(columns={"iso_a3": "CountryCode", "Country": "Name", "currency_code": "Currency_id"})

try:
    dfCountries.to_sql("Country", con,if_exists="append", index=False)
except Exception as e:
    print(e)

In [17]:
#### create image folders from db ###


cursor = con.cursor()
cursor.execute("select CountryCode from Country")
rows = cursor.fetchall()


for row in rows:
    countryCode = row[0]
        
    if not os.path.exists("../country-images/" + countryCode):
        os.makedirs("../country-images/" + countryCode)

In [25]:
### insert bigmac values ###

dfbm = df[["iso_a3", "local_price", "dollar_price", "date"]].rename(columns={"iso_a3":"Country_id", "date": "Date", "local_price":"LocalPrice","dollar_price":"USDPrice"})
dfbm["Source"] = "Economist"
try:
    dfbm.to_sql("BigMacPrice", con,if_exists="append", index=False)
except Exception as e:
    print(e)



UNIQUE constraint failed: BigMacPrice.Country_id, BigMacPrice.Date
